In [1]:
#pip install aix360

In [2]:
import os
import numpy as np
import pandas as pd
import sklearn
import xgboost
import shap
import lime
from aix360.algorithms.protodash import ProtodashExplainer
import seaborn as sb
import matplotlib.pyplot as plt
import pickle

ModuleNotFoundError: No module named 'aix360'

In [ ]:
heloc_data = pd.read_csv('heloc_dataset_v1.csv')

In [ ]:
heloc_data.head(5)

In [ ]:
heloc_data.describe()

In [ ]:
heloc_data.info()

## create correlation matrix

In [ ]:
heloc_data.RiskPerformance = heloc_data.RiskPerformance.replace(to_replace = ['Good', 'Bad'], value = [0,1])

In [ ]:
# corr = 
heloc_data.corr()
#sb.heatmap(corr, cmap="Blues", annot=True)
#plt.show()

In [ ]:
#Select import features based on correlation with Y variable (RiskPerformance)

heloc_data = heloc_data[[
    'NetFractionRevolvingBurden',
    'PercentTradesWBalance',
    'PercentInstallTrades',
    'MaxDelqEver',
    'MaxDelq2PublicRecLast12M',
    'MSinceMostRecentInqexcl7days',
    'PercentTradesNeverDelq',
    'NumSatisfactoryTrades',
    'MSinceOldestTradeOpen',
    'AverageMInFile',
    'ExternalRiskEstimate',
    'RiskPerformance']]

In [ ]:
#Create X, Y datasets

In [ ]:
X = heloc_data.drop(columns  = 'RiskPerformance')

In [ ]:
X.head(5)

In [ ]:
Y = heloc_data.RiskPerformance.replace(to_replace = ['Good', 'Bad'], value = [0,1])

In [ ]:
Y.head(5)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test  = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [ ]:
X_train_array = np.array(X_train)
X_test_array = np.array(X_test)
Y_train_array = np.array(Y_train)
Y_test_array = np.array(Y_test)

In [ ]:
xgboost_model = xgboost.XGBClassifier().fit(X_train_array, Y_train_array)

In [ ]:
train_prediction = xgboost_model.predict(X_train_array)
test_prediction = xgboost_model.predict(X_test_array)

In [ ]:
score_train = sklearn.metrics.accuracy_score(Y_train, train_prediction)
score_test = sklearn.metrics.accuracy_score(Y_test, test_prediction)

In [ ]:
print('train_score, test_score: ', score_train, score_test)

In [ ]:
from aix360.algorithms.protodash import ProtodashExplainer

In [ ]:
def protodash_explain(X_train, X_test, model, idx, m):
    
# Combine the training data set and the predicted values of the model into the same array, and distinguish the samples whose prediction results are good/bad    
    X_train_array = np.array(X_train)
    X_test_array = np.array(X_test)
    pred_train = model.predict(X_train_array)
    pred_train = pred_train.reshape((pred_train.shape[0],1)) 
    z_train = np.hstack((X_train_array, pred_train)) 
    z_train_good = z_train[z_train[:,-1]==0, :]
    z_train_bad = z_train[z_train[:,-1]==1, :]
    
    # Randomly select samples to explain from the test dataset
    print("Chosen Sample:", idx)
    print("Prediction made by the model:", model.predict(X_test_array)[idx])
    print("Prediction probabilities:", model.predict_proba(X_test_array[idx].reshape(1,X_test_array.shape[1])))
    print("")
    Target = np.hstack((X_test_array[idx].reshape((1,11)), model.predict(X_test_array)[idx].reshape((1,1))))
    print("The values of the sample features to be explained are as follows：")
    print(X_test.iloc[idx])
    
    # Create Protodash interpreter
    explainer = ProtodashExplainer()
    (W, S, setValues) = explainer.explain(Target, z_train_good, m) 
    # Display the value of each prototype feature and the corresponding Weight    
    col = X_train.columns.tolist()
    col.append('RiskPerformance')
    
    if model.predict(X_test_array)[idx]==0:
        prototypes = pd.DataFrame(z_train_good[S, :])
    
    else:
        prototypes = pd.DataFrame(z_train_bad[S, :])
    
    prototypes.columns = col
    prototypes["Weight"] = np.around(W, 5)/np.sum(np.around(W, 5))
    prototypes = prototypes.transpose()
    
    # Display the Weight of each feature of each prototype
    
    if model.predict(X_test_array)[idx]==0:
        z = z_train_good[S, 0:-1] # Store chosen prototypes
    
    else:
        z = z_train_bad[S, 0:-1] # Store chosen prototypes
    
    eps = 1e-10 # Small constant defined to eliminate divide-by-zero errors
    feature_weight = np.zeros(z.shape)
    
    for i in range (z.shape[0]):
        for j in range(z.shape[1]):
            feature_weight[i, j] = np.exp(-1 * abs(Target[0, j] - z[i,j])/(np.std(z[:, j])+eps)) # Compute feature similarity in [0,1]
    
    # move weights to a dataframe to display
    
    feature_weight_table = pd.DataFrame.from_records(np.around(feature_weight.astype('double'), 2))
    feature_weight_table.columns = X_train.columns.tolist()
    feature_weight_table = feature_weight_table.transpose() 
    return prototypes,feature_weight_table

In [ ]:
protodash_explain(X_train=X_train,X_test=X_test,model=xgboost_model,idx=8,m=5)

In [ ]:
# Combine the training data set and the predicted values of the model into the same array, 
# and distinguish the samples whose prediction results are good/bad

pred_train = train_prediction.reshape((train_prediction.shape[0],1)) 
z_train = np.hstack((X_train_array, pred_train)) 
z_train_good = z_train[z_train[:,-1]==0, :]
z_train_bad = z_train[z_train[:,-1]==1, :]

In [ ]:
# Randomly select samples to explain from the test dataset
idx = 9
print("Chosen Sample:", idx)
print("Prediction made by the model:", xgboost_model.predict(X_test_array)[idx])
print("Prediction probabilities:", xgboost_model.predict_proba(X_test_array[idx].reshape(1,X_test_array.shape[1])))
print("")
Target = np.hstack((X_test_array[idx].reshape((1,11)), xgboost_model.predict(X_test_array)[idx].reshape((1,1))))
print("The values of the sample features to be explained are as follows：")
X_test.iloc[idx]

In [ ]:
# Create Protodash interpreter
explainer = ProtodashExplainer()
(W, S, setValues) = explainer.explain(Target, z_train_good, m=5) 

In [ ]:
col = X_train.columns.tolist()
col.append('RiskPerformance')
col

In [ ]:

# Display the value of each prototype feature and the corresponding Weight
prototypes = pd.DataFrame(z_train_good[S, :])
prototypes.columns = col
prototypes["Weight"] = np.around(W, 5)/np.sum(np.around(W, 5))
prototypes.transpose()

In [ ]:
# Display the Weight of each feature of each prototype

z = z_train_good[S, 0:-1] # Store chosen prototypes
eps = 1e-10 # Small constant defined to eliminate divide-by-zero errors
feature_weight = np.zeros(z.shape)
for i in range (z.shape[0]):
    for j in range(z.shape[1]):
        feature_weight[i, j] = np.exp(-1 * abs(Target[0, j] - z[i,j])/(np.std(z[:, j])+eps)) # Compute feature similarity in [0,1]
                
# move wts to a dataframe to display

feature_weight_table = pd.DataFrame.from_records(np.around(feature_weight.astype('double'), 2))
feature_weight_table.columns = X_train.columns.tolist()
feature_weight_table.transpose() 

In [ ]:
import shap
import lime
shap.initjs()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score,roc_auc_score

In [ ]:
X_train,X_test,y_train,Y_test = train_test_split(X,Y,test_size=0.2, random_state=0)
X_train_array = np.array(X_train)
X_test_array = np.array(X_test)
Y_train_array = np.array(Y_train)
Y_test_array = np.array(Y_test)

In [ ]:
xgb_final = xgboost.XGBClassifier(tree_method = 'hist',
                         n_estimators = 800,
                         min_child_weight = 6,
                         max_depth = 2,
                         gamma = 0,
                         eta = 0.4,
                         early_stop=10,
                         random_state = 42)
xgb_final.fit(X_train_array, Y_train_array)

In [ ]:
# Model evaluation

def model_eval(model, title, test_features, test_labels):
    scores = pd.DataFrame()
    predictions = model.predict(test_features)
    accuracy = accuracy_score(test_labels,predictions)
    roc_auc = roc_auc_score(test_labels,predictions)
    F1 = f1_score(test_labels,predictions)
    precision = precision_score(test_labels,predictions)
    recall = recall_score(test_labels,predictions)
    scores[title] = [accuracy,roc_auc,F1,precision,recall]
    scores.index = ['Accuracy Score', 'ROC_AUC', 'F1_Score', 'Precision_Score','Recall_Score']
    return scores
train_scores = model_eval(xgb_final,"train",X_train_array,Y_train_array)
test_scores = model_eval(xgb_final, "test",X_test_array, Y_test_array)
print(train_scores)
print(test_scores)

In [ ]:
# Build an interpreter based on the XGBOOST model
explainer = shap.TreeExplainer(xgb_final)
shap_values = explainer.shap_values(X)
shap_interaction_values = explainer.shap_interaction_values(X)

In [ ]:
shap.force_plot(explainer.expected_value, shap_values[0,:], X.iloc[0,:])

In [ ]:
# Show global result attribution for multiple samples through thrust graph
shap.force_plot(explainer.expected_value, shap_values[:100,:], X.iloc[:100,:])

In [ ]:
# Display global result attribution through overview graph (bar)
shap.summary_plot(shap_values, X, plot_type="bar")

In [ ]:
shap.summary_plot(shap_interaction_values,X)

In [ ]:
# Show global result attribution through overview graph (scatter)
shap.summary_plot(shap_values, X)

In [ ]:
shap.dependence_plot("ExternalRiskEstimate", shap_values, X)
# Show attribution of results for a single feature through a dependency scatterplot

In [ ]:
shap.decision_plot(explainer.expected_value, shap_values[:1],X.iloc[1])
# Display the result attribution of a single sample through a decision road map

In [ ]:
shap.decision_plot(explainer.expected_value,shap_interaction_values[:1],X.iloc[1],feature_display_range=slice(None, -20, -1))

# Display the result attribution of a single sample (including the interaction between features) through a decision path diagram

In [ ]:
feature_names=['ExternalRiskEstimate', 'MSinceOldestTradeOpen',
       'MSinceMostRecentTradeOpen', 'AverageMInFile', 'NumSatisfactoryTrades',
       'NumTrades60Ever2DerogPubRec', 'NumTrades90Ever2DerogPubRec',
       'PercentTradesNeverDelq', 'MSinceMostRecentDelq',
       'MaxDelq2PublicRecLast12M', 'MaxDelqEver', 'NumTotalTrades',
       'NumTradesOpeninLast12M', 'PercentInstallTrades',
       'MSinceMostRecentInqexcl7days', 'NumInqLast6M', 'NumInqLast6Mexcl7days',
       'NetFractionRevolvingBurden', 'NetFractionInstallBurden',
       'NumRevolvingTradesWBalance', 'NumInstallTradesWBalance',
       'NumBank2NatlTradesWHighUtilization', 'PercentTradesWBalance']
feature_names = X_test.columns.tolist()
target_names=['Good','Bad']

In [ ]:
feature_names

In [ ]:
import lime
from lime import lime_tabular
lime_explainer = lime.lime_tabular.LimeTabularExplainer(X_train, feature_names=feature_names, class_names=target_names, discretize_continuous=False)
#Building LIME Interpreter Based on XGBOOST Model

In [ ]:
i = np.random.randint(0, X_test.shape[0])
print(i)
X_test.shape[0]

'''
i = int(input('Please enter obbservation id: '))
if i in str(X_test.shape[0]):
    print('id exist')
else:
    i = int(input('Please enter obbservation id: '))
'''

In [ ]:
exp = lime_explainer.explain_instance(X_test.iloc[3], xgb_final.predict_proba, num_features=11)
exp.show_in_notebook(show_table=True, show_all=False)
# Explain attribution of 1-sample results with the LIME interpreter
# Impression result attribution

In [ ]:
# save to JSON
#xgb_final.save_model("model.json")

In [ ]:
pickle.dump(xgb_final, open('model2.pkl','wb'))